### 利用PaddleHub实现变焦视频——3D照片

## 一、前言
最近，短视频平台火热的“希区柯杰”变焦视频获得极大关注，这种效果仿佛让普通的照片“活了过来”，具有十足的3D观感（如下面这样）。那我们也可以用几张照片达到这种效果吗？

## 二、实现思路
3D照片可以比作成会“动”的照片，那怎么设计才能让动的不突兀呢？  

首先，对照片按照一定比例进行裁剪，并放大与原照片一致，然后慢慢还原照片，这样照片就稍微有动起来的效果；  

其次，我们为了加强这种3D观感，需要对照片里的人物进行分割提取（使用PaddleHub模型  deeplabv3p_xception65_humanseg 实现人像分割），在背景缩放的同时进行透射变换；  

最后，我们可以添加背景音乐营造氛围。  


## 三、实现过程（最后有完整代码）

### 1. 安装必要文件
```
!pip install paddlehub -i https://mirror.baidu.com/pypi/simple
!pip install pydub	# 音频处理库
```

### 2. 导入必要模型和库
```
import numpy as np
import os
import paddlehub as hub
import cv2
import matplotlib.pyplot as plt
from pydub import AudioSegment
from moviepy.editor import *
```

### 3. 照片读取、裁剪和缩放
设置照片x轴缩放0.15，y轴缩放0.1，并根据所设置的帧数，得到每次遍历缩放的高和宽
```
files=os.listdir(filepath)
files.sort()
width,height = 3024, 4032 # 预设图片宽高
fps = 30 # 帧数设置越高，运行时间越长

# 由于后面选取的音频时间原因，单个图片需要2秒，因此总共遍历2*fps
# 读取照片，获取宽高,x轴缩放0.15，y轴缩放0.1
dheight, dwidth = 0.1 * height / (2*2*fps), 0.15 * width / (2*2*fps)
# 每次遍历，照片宽和高的缩放比例
scale_w = width/(width+2*dwidth)
scale_h = height/(height+2*dheight)
# 照片裁剪初始位置
begin_height0, begin_width0 = 0.05 * height, 0.075 * width # 上边界位置，左边界位置
begin_height1, begin_width1 = 0.95 * height, 0.925 * width # 下边界位置，右边界位置

# files中第0个文件为版本控制保存文件
for fileimg in files[1:]:
    img_=cv2.imread(filepath+'/'+fileimg)
    # 统一图片大小
    img_ = cv2.resize(img_,(width,height))
    for i in range(1,2*fps+1):
        img = img_[int(begin_height0 - i*dheight) : int(begin_height1 + i*dheight), int(begin_width0 - i*dwidth) : int(begin_width1 + i*dwidth)]
        # 恢复图片大小
        img = cv2.resize(img,(width,height), interpolation=cv2.INTER_LINEAR)
```

### 4. 定义分割函数
```
module = hub.Module(name="deeplabv3p_xception65_humanseg")

def do_seg(frame):
    result = module.segmentation(images=[frame])
    return result[0]['data']
```

### 5. 透射变换
设置初始时透射变换所需4个对应点位置 pts1，但由于照片一直处于缩放状态，为让这4个点始终相对于人像的位置不变，也需要在遍历过程中对这些点位置进行处理
```
pts1 = np.float32([[int(width/4.0/scale_w**i),int(height/8.0/scale_h**i)],\
            [int(width/4.0/scale_w**i),int(height-height/8.0/scale_h**i)],\
            [int(width-width/4.0/scale_w**i),int(height-height/8.0/scale_h**i)],\
            [int(width-width/4.0/scale_w**i),int(height/8.0/scale_h**i)]])
```
设置最终透射变换所需4个对应点位置 pts2（具体为多次尝试后的主观设定，也可以根据效果按照意愿自行更改）
```
pts2 = np.float32([[int(width/4.0/scale_w**i+width/(64*2)-width/(64.0*2*fps)*i),int(height/8.0/scale_h**i-height/(64.0*2*fps)*i)],\
            [int(width/4.0/scale_w**i+width/32-width/(16.0*2*fps)*i),int(height-height/8.0/scale_h**i)-width/64+width/(32.0*2*fps)*i],\
            [int(width-width/4.0/scale_w**i+width/(64.0*2*fps)*i),int(height-height/8.0/scale_h**i)-width/(64*2)+width/(64.0*2*fps)*i],\
            [int(width-width/4.0/scale_w**i+width/(64.0*2*fps)*i),int(height/8.0/scale_h**i)-width/(64.0*2*fps)*i]])           
```
计算变换矩阵M
```
M = cv2.getPerspectiveTransform(pts1,pts2)
```

### 6. 人像分割、缩放，以及背景缩放
```
seg = np.around(do_seg(img) / 255)
seg_mask = np.repeat(seg[:,:,np.newaxis], 3, axis=2)
front = seg_mask * img
front = cv2.warpPerspective(front,M,(width, height))
seg_mask = cv2.warpPerspective(seg_mask,M,(width, height))
res_back = (1-seg_mask) * img
```

### 7. 写入视频
```
result = res_front + res_back 
out.write(result.astype(np.uint8))
```

### 8. 音频处理，以及视频与音频合成
```
music = AudioSegment.from_wav('work/pretreatment_music/backgroun_music.wav')
clip = music[:8*1000] # 前8秒
clip.export('music.mp3', format='mp3') 
# 读取视频
video = VideoFileClip('result.mp4')
# 读取音频
audio = AudioFileClip('music.mp3')
# 设置视频的音频 
video = video.set_audio(audio)
# 保存新的视频文件
video.write_videofile('video.mp4')
```

## 四、效果改进
由于只是简单的人像透射变换以及背景缩放的组合处理，避免不了随着人像分割标记的移动，背景中浮现出原有的人像（如下图所示）  
我们可以将人像变换前与变换后的标记矩阵相减，但由于标记矩阵中的数值移动后不一样，可以先将所以的非0值变为1，再相减，这样两个矩阵相减后得到的结果就是原人像浮现的位置。  
考虑到人像分割模型可能对人像边缘处理效果不佳，我们又对人像进行适当放大，以覆盖边缘未处理部分。
```
seg_mask_copy2 = np.where(seg_mask>0,1,0)

empty_mask = seg_mask_copy1-seg_mask_copy2
empty_mask = np.where(empty_mask<=0,0,1)
empty_mask=empty_mask.astype(np.uint8) 

background_mask = seg_mask + empty_mask

background_mask = cv2.resize(background_mask,None,fx=1.05,fy=1.05, interpolation=cv2.INTER_LINEAR)
background_mask_h,background_mask_w=1.05*height,1.05*width
background_mask = background_mask[round(height*0.05/2):round(background_mask_h-height*0.05/2),round(width*0.05/2):round(background_mask_w-width*0.05/2)]

scale_r = 1.095;scale_r_=0.095
empty_mask = cv2.resize(empty_mask,None,fx=scale_r,fy=scale_r, interpolation=cv2.INTER_LINEAR)
empty_back_h,empty_back_w=scale_r*height,scale_r*width

empty_mask = empty_mask[round(height*scale_r_/2):round(empty_back_h-height*scale_r_/2),round(width*scale_r_/2):round(empty_back_w-width*scale_r_/2)]
empty_back=empty_mask*img
empty_back = cv2.resize(empty_back,None,fx=1/scale_r,fy=1/scale_r, interpolation=cv2.INTER_LINEAR)
empty_back_h,empty_back_w,_=empty_back.shape

add_h,add_w=round((height-empty_back_h)/2),round((width-empty_back_w)/2)
empty_back=np.hstack((np.zeros([empty_back_h,add_w,3]),empty_back))
empty_back=np.hstack((empty_back,np.zeros([empty_back_h,add_w,3])))
empty_back=np.vstack((np.zeros([add_h,width,3]),empty_back))
empty_back=np.vstack((empty_back,np.zeros([add_h,width,3])))

res_back = (1-background_mask) * img

res_front = front + empty_back

res_front = cv2.resize(res_front,None,fx=1.05,fy=1.05, interpolation=cv2.INTER_LINEAR)
res_front_h,res_front_w=1.05*height,1.05*width
res_front = res_front[round(height*0.05/2):round(res_front_h-height*0.05/2),round(width*0.05/2):round(res_front_w-width*0.05/2)]

result = res_front + res_back 
```

## 五、总结与展望


## 六、完整代码
```
import numpy as np
import os
import paddlehub as hub
import cv2
import matplotlib.pyplot as plt
from pydub import AudioSegment
from moviepy.editor import *

os.environ["CUDA_VISIBLE_DEVICES"]="0"

filepath = 'work/pretreatment_images'
resultname = 'result.mp4'
files=os.listdir(filepath)
files.sort()
# 设置图片宽高
width,height = 3024, 4032
fps = 30 #帧数设置越高，运行时间越长
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
out = cv2.VideoWriter(resultname, fourcc, fps, (width,height))

dheight, dwidth = 0.1 * height / (2*2*fps), 0.15 * width / (2*2*fps)
begin_height0, begin_width0 = 0.05 * height, 0.075 * width
begin_height1, begin_width1 = 0.95 * height, 0.925 * width

scale_w = width/(width+2*dwidth)
scale_h = height/(height+2*dheight)


for fileimg in files[1:]:
    img_=cv2.imread(filepath+'/'+fileimg)
    
    # 读取照片，获取宽高,x轴缩放0.15，y轴缩放0.1
    img_ = cv2.resize(img_,(width,height))

    for i in range(1,2*fps+1):
        img = img_[int(begin_height0 - i*dheight) : int(begin_height1 + i*dheight), int(begin_width0 - i*dwidth) : int(begin_width1 + i*dwidth)]
        img = cv2.resize(img,(width,height), interpolation=cv2.INTER_LINEAR)
        seg = np.around(do_seg(img) / 255)
        seg_mask = np.repeat(seg[:,:,np.newaxis], 3, axis=2)
        seg_mask_copy1 = np.where(seg_mask>0,1,0)

        pts1 = np.float32([[int(width/4.0/scale_w**i),int(height/8.0/scale_h**i)],\
            [int(width/4.0/scale_w**i),int(height-height/8.0/scale_h**i)],\
            [int(width-width/4.0/scale_w**i),int(height-height/8.0/scale_h**i)],\
            [int(width-width/4.0/scale_w**i),int(height/8.0/scale_h**i)]])
        pts2 = np.float32([[int(width/4.0/scale_w**i+width/(64*2)-width/(64.0*2*fps)*i),int(height/8.0/scale_h**i-height/(64.0*2*fps)*i)],\
            [int(width/4.0/scale_w**i+width/32-width/(16.0*2*fps)*i),int(height-height/8.0/scale_h**i)-width/64+width/(32.0*2*fps)*i],\
            [int(width-width/4.0/scale_w**i+width/(64.0*2*fps)*i),int(height-height/8.0/scale_h**i)-width/(64*2)+width/(64.0*2*fps)*i],\
            [int(width-width/4.0/scale_w**i+width/(64.0*2*fps)*i),int(height/8.0/scale_h**i)-width/(64.0*2*fps)*i]])
        # pts2 = np.float32([[int(seg_width/4.0/scale_w**i+seg_width/(64*2)-seg_width/(64.0*2*fps)*i),int(seg_height/8.0/scale_h**i+seg_height/(64*2)-seg_height/(64.0*2*fps)*i)],\
        #     [int(seg_width/4.0/scale_w**i+seg_width/32-seg_width/(16.0*2*fps)*i),int(seg_height-seg_height/8.0/scale_h**i)-seg_width/64+seg_width/(32.0*2*fps)*i],\
        #     [int(seg_width-seg_width/4.0/scale_w**i-seg_width/128+seg_width/(64.0*2*fps)*i),int(seg_height-seg_height/8.0/scale_h**i)-seg_width/(64*2)+seg_width/(64.0*2*fps)*i],\
        #     [int(seg_width-seg_width/4.0/scale_w**i-seg_width/128+seg_width/(64.0*2*fps)*i),int(seg_height/8.0/scale_h**i)+seg_width/(64*2)-seg_width/(64.0*2*fps)*i]])

        M = cv2.getPerspectiveTransform(pts1,pts2)

        front = seg_mask * img

        front = cv2.warpPerspective(front,M,(width, height))
        seg_mask = cv2.warpPerspective(seg_mask,M,(width, height))

        seg_mask_copy2 = np.where(seg_mask>0,1,0)

        empty_mask = seg_mask_copy1-seg_mask_copy2
        empty_mask = np.where(empty_mask<=0,0,1)
        empty_mask=empty_mask.astype(np.uint8) 

        background_mask = seg_mask + empty_mask

        background_mask = cv2.resize(background_mask,None,fx=1.05,fy=1.05, interpolation=cv2.INTER_LINEAR)
        background_mask_h,background_mask_w=1.05*height,1.05*width
        background_mask = background_mask[round(height*0.05/2):round(background_mask_h-height*0.05/2),round(width*0.05/2):round(background_mask_w-width*0.05/2)]

        scale_r = 1.095;scale_r_=0.095
        empty_mask = cv2.resize(empty_mask,None,fx=scale_r,fy=scale_r, interpolation=cv2.INTER_LINEAR)
        empty_back_h,empty_back_w=scale_r*height,scale_r*width

        empty_mask = empty_mask[round(height*scale_r_/2):round(empty_back_h-height*scale_r_/2),round(width*scale_r_/2):round(empty_back_w-width*scale_r_/2)]
        empty_back=empty_mask*img
        empty_back = cv2.resize(empty_back,None,fx=1/scale_r,fy=1/scale_r, interpolation=cv2.INTER_LINEAR)
        empty_back_h,empty_back_w,_=empty_back.shape

        add_h,add_w=round((height-empty_back_h)/2),round((width-empty_back_w)/2)
        empty_back=np.hstack((np.zeros([empty_back_h,add_w,3]),empty_back))
        empty_back=np.hstack((empty_back,np.zeros([empty_back_h,add_w,3])))
        empty_back=np.vstack((np.zeros([add_h,width,3]),empty_back))
        empty_back=np.vstack((empty_back,np.zeros([add_h,width,3])))

        res_back = (1-background_mask) * img
        
        res_front = front + empty_back

        res_front = cv2.resize(res_front,None,fx=1.05,fy=1.05, interpolation=cv2.INTER_LINEAR)
        res_front_h,res_front_w=1.05*height,1.05*width
        res_front = res_front[round(height*0.05/2):round(res_front_h-height*0.05/2),round(width*0.05/2):round(res_front_w-width*0.05/2)]

        result = res_front + res_back 
    # result=result.astype(np.uint8) 
    # result = result[:,:,::-1]
    # plt.scatter([pts1[0][0],pts1[1][0],pts1[2][0],pts1[3][0]],[pts1[0][1],pts1[1][1],pts1[2][1],pts1[3][1]],c='r',s=30)
    # plt.scatter([pts2[0][0],pts2[1][0],pts2[2][0],pts2[3][0]],[pts2[0][1],pts2[1][1],pts2[2][1],pts2[3][1]],c='b',s=30)
    # plt.imshow(result)
        out.write(result.astype(np.uint8))

out.release() 

music = AudioSegment.from_wav('work/pretreatment_music/backgroun_music.wav')
clip = music[:8*1000]
clip.export('music.mp3', format='mp3') 

# 读取视频
video = VideoFileClip('result.mp4')
# 读取音频
audio = AudioFileClip('music.mp3')
# 设置视频的音频 
video = video.set_audio(audio)
# 保存新的视频文件
video.write_videofile('video.mp4')
```